# EdgeTTS

https://github.com/rany2/edge-tts

edge-tts is a Python module that allows you to use Microsoft Edge's online text-to-speech service from within your Python code or using the provided edge-tts or edge-playback command.

In [2]:
%pip install edge-tts

# English Voices

* voice = "en-US-GuyNeural" (Male)
* voice = "en-US-AnaNeural" (Female)
* voice = "en-US-AndrewNeural" (Male)
* voice = "en-US-AriaNeural" (Female)
* voice = "en-US-AvaNeural" (Female)
* voice = "en-US-BrianNeural" (Male)
* voice = "en-US-ChristopherNeural" (Male)
* voice = "en-US-EmmaNeural" (Female)
* voice = "en-US-EricNeural" (Male)
* voice = "en-US-GuyNeural" (Male)
* voice = "en-US-JennyNeural" (Female)
* voice = "en-US-MichelleNeural" (Female)
* voice = "en-US-RogerNeural" (Male)
* voice = "en-US-SteffanNeural" (Male)
* voice = "en-GB-LibbyNeural" (Female)
* voice = "en-GB-MaisieNeural" (Female)
* voice = "en-GB-RyanNeural" (Male)
* voice = "en-GB-SoniaNeural" (Female)
* voice = "en-GB-ThomasNeural" (Male)
* voice = "en-AU-NatashaNeural" (Female)
* voice = "en-AU-WilliamNeural" (Male)
* voice = "en-CA-ClaraNeural" (Female)
* voice = "en-CA-LiamNeural" (Male)

In [2]:
import edge_tts
import os
import pygame
import time

async def generate_edge_tts_audio(text, file_name, voice='en-US-GuyNeural', style='newscast-formal', verbose=False, play=False, overwrite=False):
    communicate = edge_tts.Communicate(text, voice)
    # whether file exists?
    if os.path.exists(file_name):
        if overwrite:
            if verbose:
                print(f'{file_name} exists, overwriting...')
        else:
            if verbose:
                print(f'{file_name} exists, skipping...')
            return
        
    await communicate.save(file_name)
    if play:
        pygame.mixer.init()
        pygame.mixer.music.load(file_name)
        pygame.mixer.music.play()
    if verbose:
        print(f'{file_name} created')
    
    time.sleep(1.5)



voices = ["en-US-GuyNeural", "en-US-AriaNeural", "en-GB-RyanNeural", "en-GB-LibbyNeural"]
regions = ['us', 'us', 'uk', 'uk']
genders = ['male', 'female', 'male', 'female']

only_us = True
if only_us:
    voices = voices[:2]
    print(voices)

words = """
card, mark, door,short,poor,your,focus,individual,
about,
banana,
sofa,
open,
problem,
synthesis,
animal,
president,
pupil,
bacon,
lesson,
pilot,
circus,
medium,
support,
courageous,
famous,
humorous,
better,
butter,
teacher,
actor,
doctor,
editor,
beggar,
burglar,
collar,
culture,
figure,
pressure,
her,
term,
clerk,
earn,
earth,
heard,
bird,
first,
stir,
burn,
nurse,
turn,
word,
world,
worse,
"""

for word in words.strip().split(','):
    print(word)
    for i, voice in enumerate(voices):
        w = word.strip().lower()
        if len(w) > 0:
            filename = f'../audios/{w}-{regions[i]}-{genders[i]}.mp3'
            await generate_edge_tts_audio(w, filename, voice=voice, verbose=True, overwrite=True, play=True)


['en-US-GuyNeural', 'en-US-AriaNeural']
card
../audios/card_us_male.mp3 exists, overwriting...
../audios/card_us_male.mp3 created
../audios/card_us_female.mp3 exists, overwriting...
../audios/card_us_female.mp3 created
 mark
../audios/mark_us_male.mp3 created
../audios/mark_us_female.mp3 created
 door
../audios/door_us_male.mp3 created
../audios/door_us_female.mp3 created
short
../audios/short_us_male.mp3 created
../audios/short_us_female.mp3 created
poor
../audios/poor_us_male.mp3 exists, overwriting...
../audios/poor_us_male.mp3 created
../audios/poor_us_female.mp3 exists, overwriting...
../audios/poor_us_female.mp3 created
your
../audios/your_us_male.mp3 created
../audios/your_us_female.mp3 created
focus
../audios/focus_us_male.mp3 created
../audios/focus_us_female.mp3 created
individual
../audios/individual_us_male.mp3 created
../audios/individual_us_female.mp3 created

about
../audios/about_us_male.mp3 exists, overwriting...
../audios/about_us_male.mp3 created
../audios/about_us_f

error: music_mpg123: corrupt mp3 file (bad tags.)

In [ ]:
pygame.mixer.init()
pygame.mixer.music.load(OUTPUT_FILE)
pygame.mixer.music.play()

In [3]:
#American

import asyncio
import edge_tts
import pygame

TEXT = "hands"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['en-US-GuyNeural', 'en-US-MichelleNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Male').replace('MichelleNeural', 'Female').replace('en-', '').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [2]:
#American Sentence

import asyncio
import edge_tts
import pygame

TEXT = "cat,foot"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['en-US-EricNeural', 'en-US-MichelleNeural']:
    w = w.strip().replace(' ', '-')
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Male').replace('MichelleNeural', 'Female').replace('en-', '').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [ ]:
# ru-RU-DmitryNeural
# ko-KR-InJoonNeural

In [51]:
# British

import asyncio
import edge_tts
import pygame

TEXT = "speak, student, school, frustrate"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['en-GB-RyanNeural', 'en-GB-SoniaNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('RyanNeural', 'Male').replace('SoniaNeural', 'Female').replace('en-GB-', 'uk-').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [49]:
#Korean

import asyncio
import edge_tts
import pygame

TEXT = "photo, video"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['ko-KR-InJoonNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('InJoonNeural', 'kr').replace('SoniaNeural', 'Female').replace('ko-KR', 'ko').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!


In [55]:
#Chinese

import asyncio
import edge_tts
import pygame

TEXT = "room"
Wordlist = TEXT.split(",")

for w in Wordlist:
  # for VOICE in ['en-US-GuyNeural', 'en-US-JennyNeural', 'en-GB-RyanNeural', 'en-GB-SoniaNeural']:
  for VOICE in ['zh-CN-liaoning-XiaobeiNeural']:
    w = w.strip()
    # OUTPUT_FILE = f"{w}-{VOICE.replace('EricNeural', 'Guy-Male').replace('JennyNeural', 'Jenny-Female').replace('RyanNeural', 'Ryan-Male').replace('SoniaNeural', 'Sonia-Female').lower()}.mp3"
    OUTPUT_FILE = f"{w}-{VOICE.replace('liaoning-XiaobeiNeural', 'zh').replace('SoniaNeural', 'Female').replace('zh-CN', 'cn').lower()}.mp3"
    communicate = edge_tts.Communicate(w, VOICE)
    await communicate.save(OUTPUT_FILE) 

print("Files created!") 

Files created!
